In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./test.csv
./train.csv
./gender_submission.csv
./titanic.ipynb


In [2]:
import warnings
import warnings
warnings.filterwarnings('ignore')

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import PCA

df = pd.read_csv('./train.csv')

df_train = df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1).dropna()
df_train_dummy = pd.get_dummies(df_train)

df_test = pd.read_csv('./test.csv').drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
df_test_dummy = pd.get_dummies(df_test).fillna(0)

X = df_train_dummy.drop(['Survived'], axis=1)
#pca = PCA(n_components=5)
#pca_train = pca.fit_transform(X)
#pca_test = pca.fit_transform(df_test_dummy)

y = df_train_dummy['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

lr = LinearRegression()
lr.fit(X_train, y_train)
print('The score of linear regression is {}'.format(lr.score(X_test, y_test)))

xgb_classifer = xgb.XGBClassifier()

xgb_classifer.fit(X_train, y_train)
scores = cross_val_score(xgb_classifer, X_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgb_classifer, X_train, y_train, cv=kfold)
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

y_pred = xgb_classifer.predict(X_test)
print('The accuracy_score of XGBoost is {}'.format(accuracy_score(y_test, y_pred)))
cm = confusion_matrix(y_test,y_pred)
print(cm)
cr = classification_report(y_test, y_pred)
print(cr)

test_pred = xgb_classifer.predict(df_test_dummy)

df_submission = pd.read_csv('gender_submission.csv')
submission = pd.concat([df_submission['PassengerId'], pd.DataFrame(test_pred)], axis=1)
submission.columns = ['PassengerId','Survived']
submission.to_csv('submission.csv', index=False)

The score of linear regression is 0.35932595327104866
[00:12:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:12:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:12:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:12:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicit

In [31]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=10)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print('RandomForest score: {}'.format(accuracy_score(y_test, y_pred)))

test_pred = rfc.predict(df_test_dummy)

df_submission = pd.read_csv('gender_submission.csv')
submission = pd.concat([df_submission['PassengerId'], pd.DataFrame(test_pred)], axis=1)
submission.columns = ['PassengerId','Survived']
submission.to_csv('submission_rfc.csv', index=False)

RandomForest score: 0.8146067415730337


In [27]:
y_pred

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0])